In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [23]:
df = pd.read_csv('../generated_mem_data_with_anomaly.csv')
df['date'] = pd.to_datetime(df['date'])

In [24]:
df

,mem_used,mem_specs,date,anomaly
0,1.000000,16777216.0,2023-07-25 00:03:06.466,1
1,0.862886,16777216.0,2023-07-25 00:08:06.466,1
2,0.682798,16777216.0,2023-07-25 00:13:06.466,0
3,0.959150,16777216.0,2023-07-25 00:18:06.466,1
4,1.000000,16777216.0,2023-07-25 00:23:06.466,1
...,...,...,...,...
17851,0.066638,16777216.0,2023-09-24 23:38:08.122,0
17852,0.228046,16777216.0,2023-09-24 23:43:08.122,0
17853,0.637319,16777216.0,2023-09-24 23:48:08.122,1
17854,0.351833,16777216.0,2023-09-24 23:53:08.122,0


In [25]:
num_lags = 2

#add lag
for i in range(1, num_lags+1):
    df['lag_{}'.format(i)] = df['mem_used'].shift(i)
  
df = df.dropna()

In [26]:
df

,mem_used,mem_specs,date,anomaly,lag_1,lag_2
2,0.682798,16777216.0,2023-07-25 00:13:06.466,0,0.862886,1.000000
3,0.959150,16777216.0,2023-07-25 00:18:06.466,1,0.682798,0.862886
4,1.000000,16777216.0,2023-07-25 00:23:06.466,1,0.959150,0.682798
5,0.719119,16777216.0,2023-07-25 00:28:06.466,0,1.000000,0.959150
6,0.746560,16777216.0,2023-07-25 00:33:06.466,0,0.719119,1.000000
...,...,...,...,...,...,...
17851,0.066638,16777216.0,2023-09-24 23:38:08.122,0,0.274351,0.302305
17852,0.228046,16777216.0,2023-09-24 23:43:08.122,0,0.066638,0.274351
17853,0.637319,16777216.0,2023-09-24 23:48:08.122,1,0.228046,0.066638
17854,0.351833,16777216.0,2023-09-24 23:53:08.122,0,0.637319,0.228046


In [27]:
X = df[['mem_used', 'lag_1', 'lag_2']]
y = df['anomaly']

In [28]:
X_train = X[:int(0.7*len(X))]
X_test = X[int(0.7*len(X)):]
y_train = y[:int(0.7*len(y))]
y_test = y[int(0.7*len(y)):]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


C:\Users\HP\AppData\Local\Temp\ipykernel_24544\3393326467.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = y[:int(0.7*len(y))]
C:\Users\HP\AppData\Local\Temp\ipykernel_24544\3393326467.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = y[int(0.7*len(y)):]


In [29]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(3, 1), return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
391/391 [==============================] - 8s 10ms/step - loss: 0.2601 - accuracy: 0.9423 - val_loss: 0.2562 - val_accuracy: 0.9306
Epoch 2/20
391/391 [==============================] - 3s 7ms/step - loss: 0.2207 - accuracy: 0.9423 - val_loss: 0.2430 - val_accuracy: 0.9306
Epoch 3/20
391/391 [==============================] - 3s 7ms/step - loss: 0.2113 - accuracy: 0.9423 - val_loss: 0.2373 - val_accuracy: 0.9306
Epoch 4/20
391/391 [==============================] - 3s 7ms/step - loss: 0.1925 - accuracy: 0.9423 - val_loss: 0.2278 - val_accuracy: 0.9306
Epoch 5/20
391/391 [==============================] - 3s 7ms/step - loss: 0.1698 - accuracy: 0.9424 - val_loss: 0.1929 - val_accuracy: 0.9311
Epoch 6/20
391/391 [==============================] - 3s 7ms/step - loss: 0.1658 - accuracy: 0.9423 - val_loss: 0.1940 - val_accuracy: 0.9311
Epoch 7/20
391/391 [==============================] - 3s 7ms/step - loss: 0.1601 - accuracy: 0.9433 - val_loss: 0.1802 - val_accuracy: 0.9334
Epoch

In [32]:
model.evaluate(X_test, y_test)

168/168 [==============================] - 1s 4ms/step - loss: 0.1898 - accuracy: 0.9317


[0.1897818148136139, 0.9316781759262085]

In [33]:
model.predict([
  [0.53, 0.53, 0.20],
  [0.48, 0.53, 0.53],
  [0.60, 0.48, 0.53],
  [0.53, 0.60, 0.48],
  [0.24, 0.53, 0.60],
  [0.83, 0.24, 0.53],
  [0.24, 0.83, 0.24],
  [0.45, 0.24, 0.83],
  [1, 1, 1]
])

1/1 [==============================] - 1s 995ms/step


array([[4.8473790e-02],
       [2.3698246e-03],
       [2.0730928e-02],
       [4.1508200e-03],
       [1.9318149e-04],
       [5.9249759e-01],
       [2.8111436e-04],
       [2.9007157e-03],
       [1.6630528e-02]], dtype=float32)

In [34]:
model.save('memory_model.h5')

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
